In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import os
from os import listdir
from os.path import isfile, join
import random
import numpy as np

seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [2]:
from PIL import Image

class train_dataset(Dataset):

    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.img_path = []
        self.labels = []
        
        for l in range(5):
            labelPath = join(dataset_path, str(l))
            file_names = [f for f in listdir(labelPath) if isfile(join(labelPath, f))]
            
            for file_name in file_names:
                self.img_path.append([join(labelPath, file_name), file_name])
                self.labels.append(l)

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, index):
        path, name = self.img_path[index]
        image = Image.open(path)
        if self.transform:
            image = self.transform(image)
        return image, self.labels[index]


class test_dataset(Dataset):

    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.img_path = []
        file_names = [
            f for f in listdir(dataset_path) if isfile(join(dataset_path, f))
        ]
        file_names.sort()
        for file_name in file_names:
            self.img_path.append([join(dataset_path, file_name), file_name])

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, index):
        path, name = self.img_path[index]
        image = Image.open(path)
        if self.transform:
            image = self.transform(image)
        return image, name


In [3]:
class VGG16(nn.Module):
    def __init__(self, num_classes=100):
        super(VGG16, self).__init__()
        self.cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
        self.cnn = self.build_layers()
        self.fc = nn.Sequential(nn.Linear(512, 1024, bias=False),
                                nn.BatchNorm1d(1024), nn.ReLU(True),
                                nn.Linear(1024, 1024, bias=False),
                                nn.BatchNorm1d(1024), nn.ReLU(True),
                                nn.Linear(1024, num_classes))

    def build_layers(self):
        layers = []
        in_channels = 3
        for c in self.cfg:
            if c == 'M':
                layers += [nn.MaxPool2d(2, 2)]
            else:
                layers += [nn.Conv2d(in_channels, c, 3, padding=1, bias=False),
                           nn.BatchNorm2d(c),
                           nn.ReLU(True)]
                in_channels = c
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [4]:
class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1) -> None:
    super(ResidualBlock, self).__init__()
    self.block = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(True),
      nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1, bias=False),
      nn.BatchNorm2d(out_channels)      
    )
    self.short_cut = nn.Sequential()
    if stride != 1 or in_channels != out_channels:
      self.short_cut = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
        nn.BatchNorm2d(out_channels)
      )
    self.relu = nn.ReLU(True)
      
  def forward(self, x):
    out = self.block(x)
    out += self.short_cut(x)
    out =  self.relu(out)
    return out
  

class ResNet18(nn.Module):
  def __init__(self, num_classes) -> None:
    super(ResNet18, self).__init__()
    self.cfg = []
    self.in_channels = 64
    self.conv1 = nn.Sequential(
      nn.Conv2d(3, 64, 3, padding=1, bias=False),
      nn.BatchNorm2d(64),
      nn.ReLU(True)
    )
    self.conv2 = self.build_layers(64, 2, 1)
    self.conv3 = self.build_layers(128, 2, 2)
    self.conv4 = self.build_layers(256, 2, 2)
    self.conv5 = self.build_layers(512, 2, 2)
    self.fc = nn.Sequential(
      nn.Linear(512, num_classes)
    )
    self.average_pooling = nn.AdaptiveAvgPool2d(1)
  
  def build_layers(self, channels, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(ResidualBlock(self.in_channels, channels, stride))
      self.in_channels = channels
    return nn.Sequential(*layers)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.conv5(x)
    x = self.average_pooling(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x
  
class ResNet34(nn.Module):
  def __init__(self, num_classes) -> None:
    super(ResNet34, self).__init__()
    self.cfg = []
    self.in_channels = 64
    self.conv1 = nn.Sequential(
      nn.Conv2d(3, 64, 3, padding=1, bias=False),
      nn.BatchNorm2d(64),
      nn.ReLU(True)
    )
    self.conv2 = self.build_layers(64, 3, 1)
    self.conv3 = self.build_layers(128, 4, 2)
    self.conv4 = self.build_layers(256, 6, 2)
    self.conv5 = self.build_layers(512, 3, 2)
    self.fc = nn.Sequential(
      nn.Linear(512, num_classes)
    )
    self.average_pooling = nn.AdaptiveAvgPool2d(1)
  
  def build_layers(self, channels, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(ResidualBlock(self.in_channels, channels, stride))
      self.in_channels = channels
    return nn.Sequential(*layers)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.conv5(x)
    x = self.average_pooling(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

In [5]:
import torch.nn.functional as F

class identity_block(nn.Module):

    def __init__(self, num_filters, in_channel):
        super(identity_block, self).__init__()
        self.f1, self.f2, self.f3 = num_filters
        self.conv1 = nn.Conv2d(in_channel,
                               self.f1,
                               kernel_size=(1, 1),
                               bias=False)
        self.batchnorm1 = nn.BatchNorm2d(self.f1)
        self.conv2 = nn.Conv2d(self.f1,
                               self.f2,
                               kernel_size=(3, 3),
                               padding='same',
                               bias=False)
        self.batchnorm2 = nn.BatchNorm2d(self.f2)
        self.conv3 = nn.Conv2d(self.f2,
                               self.f3,
                               kernel_size=(1, 1),
                               bias=False)
        self.batchnorm3 = nn.BatchNorm2d(self.f3)

    def forward(self, x):
        x_shortcut = x
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.batchnorm3(self.conv3(x))
        x += x_shortcut
        x = F.relu(x)
        return x


class convolutional_block(nn.Module):

    def __init__(self, num_filters, in_channel, strides):
        super(convolutional_block, self).__init__()
        self.f1, self.f2, self.f3 = num_filters
        self.conv1 = nn.Conv2d(in_channel,
                               self.f1,
                               kernel_size=(1, 1),
                               stride=strides,
                               bias=False)
        self.batchnorm1 = nn.BatchNorm2d(self.f1)
        self.conv2 = nn.Conv2d(self.f1,
                               self.f2,
                               kernel_size=(3, 3),
                               padding='same',
                               bias=False)
        self.batchnorm2 = nn.BatchNorm2d(self.f2)
        self.conv3 = nn.Conv2d(self.f2,
                               self.f3,
                               kernel_size=(1, 1),
                               bias=False)
        self.batchnorm3 = nn.BatchNorm2d(self.f3)
        self.conv_shortcut = nn.Conv2d(in_channel,
                                       self.f3,
                                       kernel_size=(1, 1),
                                       stride=strides,
                                       bias=False)
        self.batchnorm_shortcut = nn.BatchNorm2d(self.f3)

    def forward(self, x):
        x_shortcut = x
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.batchnorm3(self.conv3(x))
        x_shortcut = self.batchnorm_shortcut(self.conv_shortcut(x_shortcut))
        x += x_shortcut
        x = F.relu(x)
        return x

class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.num_classes = num_classes
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3),
                      stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # nn.MaxPool2d(2, 2)
        )
        self.layer2 = nn.Sequential(
            convolutional_block([64, 64, 256], 64, 1),
            identity_block([64, 64, 256], 256),
            identity_block([64, 64, 256], 256)
        )
        self.layer3 = nn.Sequential(
            convolutional_block([128, 128, 512], 256, 2),
            identity_block([128, 128, 512], 512),
            identity_block([128, 128, 512], 512),
            identity_block([128, 128, 512], 512)
        )
        self.layer4 = nn.Sequential(
            convolutional_block([256, 256, 1024], 512, 2),
            identity_block([256, 256, 1024], 1024),
            identity_block([256, 256, 1024], 1024),
            identity_block([256, 256, 1024], 1024),
            identity_block([256, 256, 1024], 1024),
            identity_block([256, 256, 1024], 1024)
        )
        self.layer5 = nn.Sequential(
            convolutional_block([512, 512, 2048], 1024, 2),
            identity_block([512, 512, 2048], 2048),
            identity_block([512, 512, 2048], 2048)
        )
        self.out_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, num_classes),
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.out_layer(x)
        return x


In [6]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG16(30).to(device)
# model = ResNet18(5).to(device)
# model = ResNet34(5).to(device)
# model = ResNet50(5).to(device)

learning_rate = 0.1
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                 milestones=[60, 120, 160],
                                                 gamma=0.2)
criterion = nn.CrossEntropyLoss()
epochs = 250

batch_size = 512
train_transforms = transforms.Compose([
    # data augmentation
    transforms.Pad(4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_data = train_dataset('hw2/CIFAR100_testImage/Training_data',
                           transform=train_transforms)
valid_data = train_dataset('hw2/CIFAR100_testImage/Testing_data',
                           transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                  (0.2023, 0.1994, 0.2010))
                         ]))

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_data,
                          batch_size=512,
                          shuffle=False,
                          num_workers=4)

In [7]:
from tqdm.auto import tqdm

f = open('project2_cifar_with_data_aug_and_lr.txt', 'w')

for epoch in range(epochs):
    model.train()
    loop = tqdm(enumerate(train_loader), total=len(train_loader))
    total = 0
    train_accuracy = 0.0
    train_loss = 0.0
    for i, (image, label) in loop:
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        predict = outputs.argmax(dim=-1)
        train_accuracy += (predict == label).sum().item()
        total += label.shape[0]

        loop.set_description('Epoch [{}/{}]'.format(epoch, epochs))
        loop.set_postfix({
                'Train_Loss': loss.item(),
                'Train_Accuracy': '{:.2f}%'.format(100 * (train_accuracy / total))})


    f.write('{:.2f} {:.2f} '.format(loss.item(), 100 * (train_accuracy / total)))

    model.eval()
    val_accuracy = 0.0
    val_loss = 0.0
    total = 0
    with torch.no_grad():
        for val_image, val_label in valid_loader:
            val_image, val_label = val_image.to(device), val_label.to(device)
            outputs = model(val_image)
            loss = criterion(outputs, val_label)

            predict = outputs.argmax(dim=-1)
            val_accuracy += (predict == val_label).sum().item()
            total += val_label.shape[0]

        print('Val_Loss: {:.2f}   Val_Accuracy: {:.2f}'.format(
           val_loss / total, 100 * (val_accuracy / total)))

        f.write('{:.2f} {:.2f}\n'.format(loss.item(), 100 * (val_accuracy / total)))



    scheduler.step()

f.close()

Epoch [0/250]: 100%|██████████| 5/5 [00:02<00:00,  1.78it/s, Train_Loss=15.5, Train_Accuracy=18.32%]


Val_Loss: 0.00   Val_Accuracy: 20.00


Epoch [1/250]: 100%|██████████| 5/5 [00:01<00:00,  4.14it/s, Train_Loss=2.86, Train_Accuracy=18.44%]


Val_Loss: 0.00   Val_Accuracy: 20.00


Epoch [2/250]: 100%|██████████| 5/5 [00:01<00:00,  4.13it/s, Train_Loss=1.65, Train_Accuracy=22.08%]


...

Val_Loss: 0.00   Val_Accuracy: 66.40


Epoch [248/250]: 100%|██████████| 5/5 [00:01<00:00,  4.05it/s, Train_Loss=0.835, Train_Accuracy=62.48%]


Val_Loss: 0.00   Val_Accuracy: 66.40


Epoch [249/250]: 100%|██████████| 5/5 [00:01<00:00,  4.11it/s, Train_Loss=0.891, Train_Accuracy=62.20%]


Val_Loss: 0.00   Val_Accuracy: 67.00


In [8]:
# test_data = test_dataset('2', 
#                          transform=transforms.Compose([
#                              transforms.ToTensor(),
#                              transforms.Normalize((0.4914, 0.4822, 0.4465),
#                                                   (0.2023, 0.1994, 0.2010))
#                          ]))
# test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

# with open('711183116.txt', 'w') as f:
#     for img, file_name in test_loader:
#         # print(img.shape)
#         img = img.to(device)
#         model.eval()
#         with torch.no_grad():
#             output = model(img)
#             predict = output.argmax(dim=-1)

#             print('{} {}\n'.format(file_name[0].split('.')[0], predict.item()))
#             f.write('{} {}\n'.format(file_name[0].split('.')[0], predict.item()))
